Многомерный временной ряд содержит две или более переменных. См. пример ниже. Часто эти наборы данных изучаются с целью прогнозирования одной или нескольких из этих переменных.

Большинство моделей прогнозирования основаны на авторегрессии. Это равносильно решению задачи регрессии обучения с учителем. Будущие значения ряда являются целевыми переменными. Входные независимые переменные представляют собой недавние прошлые значения каждой переменной.

Авторегрессия работает при одном основном предположении. Эти недавние прошлые значения содержат достаточно информации о будущем. Но это может быть неправдой.

Вы можете попытаться извлечь больше информации из последних данных. Например, скользящая сводная статистика полезна для описания недавней динамики.

### Автоматизированная разработка функций

Разработка функций включает в себя извлечение и изучение независимых переменных. Это ключевой этап в любом проекте по науке о данных. Качество функций является центральным аспектом производительности моделей. Из-за этого специалисты по данным тратят много времени на этот процесс.

Тем не менее, проектирование функций часто является процессом ad-hoc. Специалисты по данным создают функции на основе своих знаний и опыта в предметной области. Таким образом, автоматизация части этого процесса желательна для практиков.

# Разработка признаков для многомерных временных рядов

### Чтение данных

В качестве примера мы будем использовать многомерный временной ряд, собранный с умного буя [1]. Этот буй находится на побережье Ирландии. Он фиксирует 9 переменных, связанных с состоянием океана. К ним относятся, среди прочего , температура моря, высота волн и скорость морской воды . На рисунке 1 выше показан график за первый месяц 2022 года.



In [8]:
info={}
import pandas as pd 

# пропуская вторую строку, устанавливая столбец времени как столбец datetime 
# набор данных доступен здесь: https://github.com/vcerqueira/blog/tree/main/data 
# df = pd.read_csv(
#     "https://raw.githubusercontent.com/vasiache/blog/main/data/smart_buoy.csv"
#                   skiprows=[1], 
#                   parse_dates=['time'], sep=",") 
smart_buoy = pd.read_csv('data/smart_buoy.csv', 
                   skiprows=[1], 
                   parse_dates=['time'], sep=",") 
# установка времени в качестве индекса 
smart_buoy.set_index('time', inplace=True)
print ('load complete')



load complete


In [9]:
from utils.utils import plotDFs

In [10]:
smart_buoy.select_dtypes(include=['int', 'float'])

,PeakPeriod,PeakDirection,UpcrossPeriod,SignificantWaveHeight,SeaTemperature,Hmax,THmax,MeanCurDirTo,MeanCurSpeed
time,,,,,,,,,
2022-01-01 00:00:00+00:00,4.17,169.05495,3.96,154.0,10.16,272.0,4.91,0.351648,0.216
2022-01-01 00:05:00+00:00,NaN,NaN,NaN,NaN,10.16,NaN,NaN,NaN,NaN
2022-01-01 00:10:00+00:00,NaN,NaN,NaN,NaN,10.11,NaN,NaN,1.934066,0.170
2022-01-01 00:15:00+00:00,NaN,NaN,NaN,NaN,10.11,NaN,NaN,NaN,NaN
2022-01-01 00:20:00+00:00,NaN,NaN,NaN,NaN,10.11,NaN,NaN,1.054945,0.138
...,...,...,...,...,...,...,...,...,...
2022-11-23 10:35:00+00:00,NaN,NaN,NaN,NaN,10.94,NaN,NaN,NaN,NaN
2022-11-23 10:40:00+00:00,NaN,NaN,NaN,NaN,10.94,NaN,NaN,53.010990,0.256
2022-11-23 10:45:00+00:00,NaN,NaN,NaN,NaN,10.94,NaN,NaN,NaN,NaN


In [11]:
smart_buoy.fillna(method='bfill', inplace=True)
smart_buoy.interpolate(inplace=True)
# упрощение имен столбцов 
smart_buoy.columns = ['station_id', 
    'PeakP', 'PeakD', 'Upcross', 
    'SWH', 'SeaTemp', 'Hmax', 'THmax', 
    'MCurDir', 'MCurSpd' 
]
smart_buoy=smart_buoy.select_dtypes(include=['int', 'float'])
# повторная выборка в дневные данные, чтобы уменьшить обхем данных
smart_buoy = smart_buoy.resample('D').mean () 

In [7]:
plotDFs([smart_buoy])

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
# https://github.com/vcerqueira/blog/blob/main/src/tde.py
from models.tde import time_delay_embedding
target_var = 'SWH'
colnames = smart_buoy.columns.tolist()
# create data set with lagged features using time delay embedding
smart_buoy_ds = []
for col in smart_buoy: 
    col_df = time_delay_embedding(smart_buoy[col], n_lags=24, horizon=12) 
    smart_buoy_ds.append(col_df)
    # concatenating all variables
    smart_buoy_df = pd.concat(smart_buoy_ds, axis=1).dropna()

In [6]:
# определение цели (Y) и независимых переменных (X) 
Predictor_variables = smart_buoy_df.columns.str.contains('\(t\-') 
target_variables = smart_buoy_df.columns.str.contains(f'{target_var}\(t\+') 
X = smart_buoy_df.iloc[:, Predictor_variables] 
Y = smart_buoy_df.iloc[:, target_variables] 
print ('prepare complete')
print (f'target = {target_var}')

prepare complete
target = SWH


In [7]:
# разделение обучения/тестирования 
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X, Y, test_size=0.3, shuffle=False) 

# подгонка модели lgbm без разработки признаков 
model_wo_fe = MultiOutputRegressor(LGBMRegressor()) 
model_wo_fe.fit(X_tr, Y_tr) 

# получение прогнозов для набора тестов 
preds_wo_fe = model_wo_fe.predict(X_ts) 

# вычисление ошибки MAPE 
info={'base_MultiOutputRegressor':mape(Y_ts, preds_wo_fe)}
print(f' base_MultiOutputRegressor {mape(Y_ts, preds_wo_fe)}')  

 base_MultiOutputRegressor 0.8519350485055593


Во-первых, временной ряд превращается в авторегрессионную задачу. Это делается с помощью функции time_delay_embedding . Прогностическая цель состоит в том, чтобы спрогнозировать следующие 12 значений SWH ( Horizon=12 ). Независимые переменные — это последние 24 значения каждой переменной в ряду ( n_lags=24 ).

LightGBM обучается для каждого горизонта прогнозирования с использованием прямого подхода. Прямой метод является популярным подходом для прогнозирования на несколько шагов вперед . Он реализован в scikit-learn под названием MultiOutputRegressor.

Приведенный выше код строит и тестирует авторегрессионную модель. Независимые переменные включают только недавние прошлые значения каждой переменной. Это приводит к средней абсолютной ошибке в процентах 0,238 . Можно ли улучшить этот показатель с помощью разработки функций.

Два подхода к извлечению признаков из многомерных временных рядов:

- Извлечение одномерных признаков. Вычисление скользящей статистики каждой переменной. Например, скользящее среднее можно использовать для сглаживания ложных наблюдений;
- Извлечение двумерных признаков. Вычисление скользящей статистики пар переменных для суммирования их взаимодействия. Например, скользящая ковариация между двумя переменными.


### Одномерное извлечение признаков

Можно суммировать недавние прошлые значения каждой переменной. Например, вычисление скользящего среднего для суммирования последнего уровня. Или скользящую дисперсию, чтобы понять недавнюю степень дисперсии.

In [8]:
import numpy as np

SUMMARY_STATS = {
    'mean': np.mean,
    'sdev': np.std,
}

univariate_features = {}

for col in X.columns:
    # get lags for that column
    X_col = X.filter(like=col)
    # compute the stats for all columns in the current batch, across rows
    stats = np.apply_along_axis(lambda x: [func(x) for func in SUMMARY_STATS.values()], axis=1, arr=X_col)
    # unpack the computed stats into individual feature columns
    for idx, name in enumerate(SUMMARY_STATS.keys()):
        univariate_features[f'{col}_{name}'] = stats[:, idx]

# объединить функции в pd.DF 
univariate_features_df = pd.DataFrame(univariate_features)
print(f' univariate_features_df complete') 


 univariate_features_df complete


In [9]:
# объединение всех функций с задержками 
# X_with_features = pd.concat([X, univariate_features_df, bivariate_features_df], axis=1) 
# X_with_features = X.merge(univariate_features_df, left_index=True, right_index=True)
univariate_features_df.index=X.index
X_with_features = pd.concat([X, univariate_features_df], axis=1) 

# разделение обучения/тестирования 
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_with_features, Y, test_size=0.3, shuffle=False ) 

# подгонка модели lgbm с инженерными функциями 
model_w_fe = MultiOutputRegressor(LGBMRegressor()) 
model_w_fe.fit(X_tr, Y_tr) 

# получение прогнозов для тестового набора 
preds_w_fe = model_w_fe.predict(X_ts) 

# вычисление ошибки MAPE 
info={'univariate_features_df':mape(Y_ts, preds_w_fe)}
print(f' univariate_features_df {mape(Y_ts, preds_w_fe)}') 
# 0,227
# X_with_features           0.8519350485055593
# univariate_features_df    0.8519350485055593


 univariate_features_df 0.8519350485055593


Можно добавить функции в словарь SUMMARY_STATS. Наличие этих функций в одном словаре делает код аккуратным.

### Извлечение двумерных признаков

Одномерная статистика упускает потенциальные взаимодействия между различными переменными. Можно получить эту информацию, используя процесс извлечения двумерных признаков.

Идея состоит в том, чтобы вычислить признаки для разных пар переменных. Суммируем совместную динамику этих пар с бинарной статистикой.

Это можно сделать двумя способами:

- Скользящая бинарная статистика. Вычислите статистику, которая принимает пары переменных в качестве входных данных. Например, скользящая ковариация или скользящая корреляция;
- Скользящее бинарное преобразование с последующей одномерной статистикой. Преобразуем пару переменных в одну переменную и суммируем эту переменную. Например, вычисление поэлементной взаимной корреляции и последующее получение ее среднего значения.

Примеры скользящей бинарной статистики включают ковариацию, корреляцию или относительную энтропию.

Есть много возможностей бинарного преобразования. Например, процентная разница, взаимная корреляция или линейная свертка между парами переменных. Затем эти преобразования суммируются со статистическими данными, такими как среднее значение или стандартное отклонение.

In [10]:
import itertools

import pandas as pd

from scipy.spatial.distance import jensenshannon
from scipy import signal
from scipy.special import rel_entr

BIVARIATE_STATS = {
    'js_div': jensenshannon,
}

BIVARIATE_TRANSFORMATIONS = {
    'corr': signal.correlate,
    'conv': signal.convolve,
    'rel_entr': rel_entr,
}

# get all pairs of variables
col_combs = list(itertools.combinations(X.columns.tolist(), 2))

from tqdm import tqdm
bivariate_features = []
# for each row
# for i, _ in X.iterrows():
for i, _ in tqdm(X.iterrows(), total=X.shape[0]):
    # feature set in the i-th time-step
    feature_set_i = {}
    for col1, col2 in col_combs:
        # features for pair of columns col1, col2

        # getting the i-th instance for each column
        x1 = X.loc[i, X.columns.str.startswith(col1)]
        x2 = X.loc[i, X.columns.str.startswith(col2)]

        # compute each summary stat
        for feat, func in BIVARIATE_STATS.items():
            feature_set_i[f'{col1}|{col2}_{feat}'] = func(x1, x2)

        # for each transformation
        for trans_f, t_func in BIVARIATE_TRANSFORMATIONS.items():

            # apply transformation
            xt = t_func(x1, x2)

            # compute summary stat
            for feat, s_func in SUMMARY_STATS.items():
                feature_set_i[f'{col1}|{col2}_{trans_f}_{feat}'] = s_func(xt)

    bivariate_features.append(feature_set_i)

bivariate_features_df = pd.DataFrame(bivariate_features, index=X.index)
print(f'bivariate_features_df complete') 

100%|██████████| 40/40 [52:07<00:00, 78.19s/it]


bivariate_features_df complete


In [11]:
# import itertools
# import pandas as pd
# import numpy as np
# import numba
# from numba import cuda
# from math import ceil
# from scipy import signal

# from scipy.spatial.distance import jensenshannon
# from scipy.special import rel_entr
# from models.feature_extraction import covariance

# BIVARIATE_STATS = {
#     'covariance': covariance,
#     'js_div': jensenshannon,
# }
# BIVARIATE_TRANSFORMATIONS = {
#     'corr': signal.correlate,
#     'conv': signal.convolve,
#     'rel_entr': rel_entr,
# }
# col_combs = list(itertools.combinations(X.columns.tolist(), 2))

# @cuda.jit
# def jit_bivariate_features(X, bivariate_features, col_combs):
#     i, _ = cuda.grid(2)

#     if i >= X.shape[0]:
#         return
#     feature_set_i = {}
#     for k in range(len(col_combs)):
#         col1, col2 = col_combs[k]
#         x1 = X[i, col1]
#         x2 = X[i, col2]
#         for feat, func in BIVARIATE_STATS.items():
#             feature_set_i[f'{col1}|{col2}_{feat}'] = func(x1, x2)
#         for trans_f, t_func in BIVARIATE_TRANSFORMATIONS.items():
#             xt = t_func(x1, x2)
#             for feat, s_func in SUMMARY_STATS.items():
#                 feature_set_i[f'{col1}|{col2}_{trans_f}_{feat}'] = s_func(xt)
#     bivariate_features[i] = feature_set_i
# X_global_mem = cuda.to_device(X.to_numpy())
# THREADS_PER_BLOCK = 32
# BLOCKS_PER_GRID_x = ceil(X.shape[0] / THREADS_PER_BLOCK)
# BLOCKS_PER_GRID_y = 1
# BLOCKS_PER_GRID = (BLOCKS_PER_GRID_x, BLOCKS_PER_GRID_y)
# bivariate_features = np.empty(X.shape[0], dtype=object)
# jit_bivariate_features[BLOCKS_PER_GRID, THREADS_PER_BLOCK](X_global_mem, bivariate_features, col_combs)
# bivariate_features_df = pd.DataFrame(bivariate_features, index=X.index[:bivariate_features.shape[0]])
# print(f'bivariate_features_df complete') 

In [12]:
# объединение всех функций с задержками 
# X_with_features = pd.concat([X, univariate_features_df, bivariate_features_df], axis=1) 
bivariate_features_df.index=X.index
X_with_features = pd.concat([X, bivariate_features_df], axis=1) 

# разделение обучения/тестирования 
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_with_features, Y, test_size=0.3, shuffle=False ) 

# подгонка модели lgbm с инженерными функциями 
model_w_fe = MultiOutputRegressor(LGBMRegressor()) 
model_w_fe.fit(X_tr, Y_tr) 

# получение прогнозов для тестового набора 
preds_w_fe = model_w_fe.predict(X_ts) 

# вычисление ошибки MAPE 
# вычисление ошибки MAPE 
info={'bivariate_features_df':mape(Y_ts, preds_w_fe)}
print(f'bivariate_features_df {mape(Y_ts, preds_w_fe)}') 
# 0,227

bivariate_features_df 0.8519350485055593


Можно добавить дополнительные преобразования или статистику. Это делается путем включения их в словари BIVARIATE_TRANSFORMATIONS или BIVARIATE_STATS.

После извлечения всех признаков надо объединить их с исходными независимыми переменными. Затем цикл обучения и тестирования аналогичный предыдущему

In [13]:
# объединение всех функций с задержками 
X_with_features = pd.concat([X, univariate_features_df, bivariate_features_df], axis=1) 
# X_with_features = pd.concat([X, univariate_features_df], axis=1) 

# разделение обучения/тестирования 
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_with_features, Y, test_size=0.3, shuffle=False ) 

# подгонка модели lgbm с инженерными функциями 
model_w_fe = MultiOutputRegressor(LGBMRegressor()) 
model_w_fe.fit(X_tr, Y_tr) 

# получение прогнозов для тестового набора 
preds_w_fe = model_w_fe.predict(X_ts) 

# вычисление ошибки MAPE 
info={'X_with_features':mape(Y_ts, preds_w_fe)}
print(f'X_with_features {mape(Y_ts, preds_w_fe)}') 
# 0,227

X_with_features 0.8519350485055593


In [14]:
def save(info=None, to_file=False, file="data/experiments"):
    if to_file:
        import json
        with open(file+'.json', 'w') as file:
            json.dump(info, file)
    else:
        pd.DataFrame(info).to_pickle(file+'.pkl')

print (info)
save(info, to_file=True, file="data/features")
X_with_features = pd.concat([X, univariate_features_df, bivariate_features_df], axis=1) 
X_with_features
pd.DataFrame(X_with_features).to_pickle('data/X_with_features'+'.df')
pd.DataFrame(univariate_features_df).to_pickle('data/univariate_features_df'+'.df')
pd.DataFrame(bivariate_features_df).to_pickle('data/bivariate_features_df'+'.df')


{'X_with_features': 0.8519350485055593}
